Gentetic Algorithm 실습
Reference : https://towardsdatascience.com/evolution-of-a-salesman-a-complete-genetic-algorithm-tutorial-for-python-6fe5d2b3ca35
1. Create population
2. Determine fitness
3. Select the mating pool
4. Breed
5. Mutate
6. Repeat

In [2]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt

In [3]:
#Create City, Fitness Class

class City:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def distance(self,city):
        xDis = abs(self.x-city.x)
        yDis = abs(self.y-city.y)
        distance = np.sqrt((xDis**2)+(yDis**2))
        return distance
    
    def __repr__(self):
        return "("+str(self.x)+","+str(self.y)+")"
    
        

In [49]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

Create Population

In [27]:
#하나의 individual route 생성하는 함수
def createRoute(cityList):
    route = random.sample(cityList,len(cityList))
    return route

In [29]:
#여러 individual route 생성하여 population 형성하는 함수
def initialPopulation(popSize,cityList):
    population = []
    for i in range(0,popSize):
        population.append(createRoute(cityList))
    return population

Determine Fitness
Fitness 이용해 rank만들기

In [51]:
def rankRoutes(population):
    fitnessResults={}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(),key = operator.itemgetter(1),reverse=True)

In [57]:
def selection(popRanked,eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked),columns = ['index','Fitness'])
    df['cum_sum']= df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0,eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0,len(popRanked)-eliteSize):
        pick = 100*random.random()
        for i in range(0,len(popRanked)):
            if pick<= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [9]:
def matingPool(population,selectionResults):
    matingpool = []
    for i in range(0,len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

Breed

ordered crossover 사용

In [64]:
def breed(parent1,parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random()*len(parent1))
    geneB = int(random.random()*len(parent1))
    startGene = min(geneA,geneB)
    endGene = max(geneA,geneB)
    
    for i in range(startGene,endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]
    child = childP1+childP2
    return child
    

In [62]:
def breedPopulation(matingpool,eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool,len(matingpool))
    for i in range(0,eliteSize):
        children.append(matingpool[i])
        
    for i in range(0,length):
        child = breed(pool[i],pool[len(matingpool)-i-1])
        children.append(child)
    return children

Mutate

In [68]:
#swamp mutate
def mutate(individual,mutationRate):
    for swapped in range(len(individual)):
        if(random.random()<mutationRate):
            swapWith = int(random.random()*len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
        return individual

In [71]:
def mutatePopulation(population,mutationRate):
    mutatedPop = []
    for ind in range(0,len(population)):
        mutatedInd = mutate(population[ind],mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [60]:
##Repeat - 반복작업!
def nextGeneration(currentGen,eliteSize,mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked,eliteSize)
    matingpool = matingPool(currentGen,selectionResults)
    children = breedPopulation(matingpool,eliteSize)
    nextGeneration = mutatePopulation(children,mutationRate)
    return nextGeneration

In [73]:
def geneticAlgorithm(population,popSize,eliteSize,mutationRate,generations):
    pop = initialPopulation(popSize,population)
    print('intital distance:'+str(1/rankRoutes(pop)[0][1]))
    
    for i in range(0,generations):
        pop = nextGeneration(pop,eliteSize,mutationRate)
        
    print('Final distance:'+str(1/rankRoutes(pop)[0][1]))
    bestRouteIndex = rankRoutes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute

In [17]:
cityList = []
for i in range(0,25) :
    cityList.append(City(x=int(random.random()*200),y=int(random.random()*200)))

In [18]:
cityList

[(154,45),
 (108,2),
 (67,96),
 (85,8),
 (151,3),
 (140,48),
 (137,89),
 (58,50),
 (80,78),
 (80,192),
 (158,61),
 (21,95),
 (105,2),
 (23,59),
 (95,23),
 (123,54),
 (159,170),
 (47,124),
 (165,125),
 (175,175),
 (97,93),
 (42,192),
 (144,9),
 (102,90),
 (135,149)]

In [74]:
geneticAlgorithm(population=cityList,popSize=100,eliteSize=20,mutationRate=0.01,generations=500)

intital distance:1980.1911101537787
Final distance:922.4737740525331


[(158,61),
 (154,45),
 (151,3),
 (144,9),
 (85,8),
 (105,2),
 (108,2),
 (95,23),
 (58,50),
 (80,78),
 (97,93),
 (135,149),
 (165,125),
 (175,175),
 (159,170),
 (80,192),
 (42,192),
 (47,124),
 (21,95),
 (23,59),
 (67,96),
 (102,90),
 (137,89),
 (123,54),
 (140,48)]